In [1]:
import os
import random

import einops
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import torch
from torch import nn
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader
from tqdm import tqdm

from koopman import utils
from koopman.autoencoder.dataset import KoopmanDataset
from koopman.autoencoder.model import KoopmanAutoencoder
from koopman.simulation.systems import NonlinearAttractor2D

In [5]:
pred_horizon = 20
batch_size = 32

data = torch.load('/home/mreich/workspaces/koopman/koopman/autoencoder/attractor/attractor_data.pt')
ts = data["ts"]
xhist = data["xhist"]
uhist = data["uhist"]
dt = data["dt"]

dataset = KoopmanDataset(xhist, uhist, ts, pred_horizon, dt)

model = KoopmanAutoencoder(
    nx=NonlinearAttractor2D.nx,
    nu=NonlinearAttractor2D.nu,
    nz=3,
    H=pred_horizon,
    params_init='eye',
    hidden_dims=[32, 32],
    activation=nn.Mish,
    use_layernorm=False,
    horizon_loss_weight=5.0,
    L1_loss_weight=0.5,
)

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
xhist, uhist = next(iter(dataloader))

xhist = xhist.clone().detach().requires_grad_(True)
uhist = uhist.clone().detach().requires_grad_(True)

xhist_flat = einops.rearrange(xhist, 'b h nx -> (b h) nx')
uhist_flat = einops.rearrange(uhist, 'b h nu -> (b h) nu')
zhist_flat = model.forward(xhist_flat)

jx, ju = model._dynamics_jacobian_norm(xhist_flat, uhist_flat, zhist_flat)

print(jx, ju)

tensor(1.9387, grad_fn=<MeanBackward1>) tensor(12.3386, grad_fn=<MeanBackward1>)


In [3]:
jx.backward()

In [ ]:
# Example network: mapping from R^10 to R^20.
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.model = nn.Sequential(
            nn.Linear(10, 32),
            nn.Mish(),
            nn.Linear(32, 32),
            nn.Mish(),
            nn.Linear(32, 32),
            nn.Mish(),
            nn.Linear(32, 20)
        )
    
    def forward(self, x):
        return self.model(x)

model = SimpleModel()

batch_size = 16
h = 10
nx = 10
nu = 4

# fwd pass
xs = torch.randn(batch_size, h+1, nx)
# xs.requires_grad_(True)

us = torch.randn(batch_size, h, nu)
# us.requires_grad_(True)

xs_flat = einops.rearrange(xs, 'b h nx -> (b h) nx')
us_flat = einops.rearrange(us, 'b h nu -> (b h) nu')
zs_flat = model(xs_flat)

B = torch.randn(20, nu)

zs = einops.rearrange(zs_flat, '(b h) nz -> b h nz', b=batch_size)

zs_of_interest = zs[:, :-1, :]
zs_of_interest_flat = einops.rearrange(zs_of_interest, 'b h nz -> (b h) nz')

zs_next_flat = 5.0 * zs_of_interest_flat + us_flat @ B.T
xs_next_flat = zs_next_flat[:, :10]

bh_size = zs_next_flat.shape[0]
vsx = torch.randn(bh_size, 10)

xs_of_interest = xs[:, :-1, :].detach().requires_grad_(True)
xs_of_interest_flat = einops.rearrange(xs_of_interest, 'b h nx -> (b h) nx')

jx = torch.autograd.grad(xs_next_flat, xs_of_interest_flat, grad_outputs=vsx, create_graph=True)

RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.